### Importing all the required libraries

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F #
import matplotlib.pyplot as plt
import numpy as np

### Set the device to run our model to CPU. If CUDA is available we can use GPU

In [ ]:
device='cpu'

### Transformers to convert the images to the required format required by the CNN.

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

### Specify the train and test dataset from the MNIST dataset

In [ ]:
trainset=torchvision.datasets.MNIST("mnist",train=True,download=True,transform=transform)

testset=torchvision.datasets.MNIST("mnist",train=False,download=True,transform=transform)

print(trainset.data.shape)
print(testset.data.shape)

### Load 128 images for training and testing purposes. Also shuffle makes sure the data is mixed and is not in sequence.

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,
                                           batch_size = 128,
                                           shuffle = True,
                                           num_workers = 0)

testloader = torch.utils.data.DataLoader(testset,
                                          batch_size = 128,
                                          shuffle = False,
                                          num_workers = 0)

### Print the batch

In [ ]:
dataiter=iter(trainloader)
images,labels=next(dataiter)
print(images.shape)
print(labels.shape)

### To display a part of the MNIST dataset

In [ ]:
def imshow(img):
    img = img / 2 + 0.5 
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))

print(''.join('%2s' % labels[j].numpy() for j in range(128)))

### Building our model

In [ ]:
nn.Conv2d(in_channels=1,
          out_channels=32,
          kernel_size=3,
          stride=1, 
          padding=1)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 12 * 12) # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()
net.to(device)

### Defining a Loss Function and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Training the model

In [ ]:
epochs = 10
epoch_log = []
loss_log = []
accuracy_log = []
for epoch in range(epochs):  
    print(f'Starting Epoch: {epoch+1}...')
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
       
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step()
        
        running_loss += loss.item()
        if i % 50 == 49:   
            correct = 0 
            total = 0 
            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, dim = 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                accuracy = 100 * correct / total
                epoch_num = epoch + 1
                actual_loss = running_loss / 50
                print(f'Epoch: {epoch_num}, Mini-Batches Completed: {(i+1)}, Loss: {actual_loss:.3f}, Test Accuracy = {accuracy:.3f}%')
                running_loss = 0.0

   
    epoch_log.append(epoch_num)
    loss_log.append(actual_loss)
    accuracy_log.append(accuracy)

print('Finished Training')

### Saving the weights file of the trained model

In [ ]:
PATH = ' D:\\mnist_cnn_net.pth'
torch.save(net.state_dict(), PATH)

### Reload the saved model

In [ ]:
net = Net()
net.to(device)
net.load_state_dict(torch.load(PATH))

### Testing the model and getting the predictions

In [ ]:
test_iter = iter(testloader)
images, labels = test_iter.next()
images = images.to(device)
labels = labels.to(device)
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ''.join('%2s' % predicted[j].cpu().numpy() for j in range(128)))

### Accuracy of the model

In [ ]:
correct = 0 
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy:.3}%')

### Plotting the Training Loss

In [ ]:
epoch_log = []
loss_log = []
accuracy_log = []

fig, ax1 = plt.subplots()
plt.title("Accuracy & Loss vs Epoch")
plt.xticks(rotation=45)

ax2 = ax1.twinx()
ax1.plot(epoch_log, loss_log, 'g-')
ax2.plot(epoch_log, accuracy_log, 'b-')

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='g')
ax2.set_ylabel('Test Accuracy', color='b')
plt.show()